In [20]:
import cv2
import numpy as np
img = cv2.imread("hough.jpg", 0)
print(type(img))
img = np.asarray(img)

#Fetching the rows and columns
rows = len(img)
cols = len(img[0])

<class 'numpy.ndarray'>


# Sobel Operator

In [21]:
#initializing Sobel Operator
gx_sobel = [[-1,-2,-1],
           [0,0,0],
           [1,2,1]]

gy_sobel = [[-1,0,1],
           [-2,0,2],
           [-1,0,1]]

x_sobel = list(map(list,zip(*gx_sobel)))
y_sobel = list(map(list,zip(*gy_sobel)))


# Applying Gradient

In [22]:
#Used to pad the image
img_x = [[0]*902 for i in range(602)]
img_y= [[0]*902 for i in range(602)]

#Finding the Gradient to distinguish the images clearly. Especially the edges of the images
#Along X Axis

for i in range(1,img.shape[0]-1):
    for j in range(1,img.shape[1]-1):
        img_x[i][j] = img[i+1][j] - img[i-1][j]  #Subtract the neighbouring pixels on the x-axis

#Along Y Axis        
for i in range(1,img.shape[0]-1):
    for j in range(1,img.shape[1]-1):
        img_y[i][j] = img[i][j+1] - img[i][j-1]  #Subtract the neighbouring pixels on the y-axis

img_x = np.asarray(img_x)
img_y = np.asarray(img_y)

cv2.imwrite('T3_Images/Gradient_X.png',(img_x))
cv2.imwrite('T3_Images/Gradient_Y.png',(img_y))


C:\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in ubyte_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in ubyte_scalars
  from ipykernel import kernelapp as app


True

# Convolution

In [23]:
#Convolution
import math
sobelgx = [[0]*img.shape[1] for i in range(img.shape[0])]    #Creating empty 2D Lists
sobelgy = [[0]*img.shape[1] for i in range(img.shape[0])]
sobel = [[0]*img.shape[1] for i in range(img.shape[0])]
sobel_add = [[0]*img.shape[1] for i in range(img.shape[0])]

#Multiplying the sobel operator with the corresponding image pixel for that patch of the image to find the covoluted pixel value for (x,y)
for x in range(1, rows-1):
    for y in range (1, cols-1):
        gx =  x_sobel[0][0] * img[x-1][y-1] + x_sobel[0][1] * img[x][y-1] + x_sobel[0][2] * img[x+1][y-1] + (x_sobel[1][0] * img[x-1][y])   + (x_sobel[1][1] * img[x][y])   + (x_sobel[1][2] * img[x+1][y]) + (x_sobel[2][0] * img[x-1][y+1]) + (x_sobel[2][1] * img[x][y+1]) + (x_sobel[2][2] * img[x+1][y+1])
        sobelgy[x-1][y-1] = gx
        gy =  (y_sobel[0][0] * img[x-1][y-1]) + (y_sobel[0][1] * img[x][y-1]) + (y_sobel[0][2] * img[x+1][y-1]) + (y_sobel[1][0] * img[x-1][y])   + (y_sobel[1][1] * img[x][y])   + (y_sobel[1][2] * img[x+1][y]) + (y_sobel[2][0] * img[x-1][y+1]) + (y_sobel[2][1] * img[x][y+1]) + (y_sobel[2][2] * img[x+1][y+1])
        sobelgx[x-1][y-1] = gy
        
        sobel[x-1][y-1] = math.sqrt((gx*gx + gy*gy))   #Combining the outputs of the Sobel-X and Sobel-Y by Root of Squared Sum.
        sobel_add[x-1][y-1] = (gx + gy)/2              #Combining the outputs of the Sobel-x and Sobel-Y by adding the values

cv2.imwrite('T3_Images/Sobel_X.png',np.asarray(sobelgx)) 
cv2.imwrite('T3_Images/Sobel_Y.png',np.asarray(sobelgy))
cv2.imwrite('T3_Images/Sobel.png',np.asarray(sobel))
cv2.imwrite('T3_Images/Sobel_add.png',np.asarray(sobel_add))


True

# Hough Transform

In [24]:
sobelgx = np.asarray(sobelgx)
sobelgy = np.asarray(sobelgy)
sobel = np.asarray(sobel)

In [25]:
def hough(x,y, theta):
    diag = np.sqrt(thres_img.shape[0]**2+thres_img.shape[1]**2)-1
    return (int)(y*math.cos(math.radians(theta)) + x*math.sin(math.radians(theta))+ diag)

In [26]:
# sobelgx = cv2.imread("red_img.jpg", 0)
# cv2.imwrite("gray_red_img.jpg",sobelgx)

# Accumulator Matrix

In [27]:
def accumulator_matrix(thres_img,theta):
    rho_max = (int)(2 * np.sqrt(thres_img.shape[0]**2+thres_img.shape[1]**2)-1)
    acc = np.zeros((theta.shape[0], rho_max))
    for x in range(thres_img.shape[0]):
        for y in range(thres_img.shape[1]):
            if thres_img[x][y] == 255:
                for t in theta:
                    rho = hough(x,y,t)
                    #print (t)
                    acc[t+int(theta.shape[0]/2)][rho] += 1
    cv2.imwrite("accumulator.jpg",acc)
    return acc

In [28]:
def max_indices(arr, k):
    '''
    Returns the indices of the k first largest elements of arr
    (in descending order in values)
    '''
    assert k <= arr.size, 'k should be smaller or equal to the array size'
    arr_ = arr.astype(float)  # make a copy of arr
    max_idxs = []
    for _ in range(k):
        max_element = np.max(arr_)
        if np.isinf(max_element):
            break
        else:
            idx = np.where(arr_ == max_element)
        max_idxs.append(idx)
        arr_[idx] = -np.inf
    return convert(np.asarray(max_idxs))

In [29]:
def convert(k):
    edge_points = []
    for i in range(k.shape[0]):
        n = k[i][0].shape[0]
        if n>1:
            for j in range(n):
                edge_points.append([k[i][0][j], k[i][1][j]])
        else:
            edge_points.append([k[i][0][0], k[i][1][0]])
    return edge_points

# Point to Edge Detection

In [30]:
def get_edge(edge_points):
    edge_img_vertical = cv2.imread("hough.jpg")
    edge_img_horizontal = cv2.imread("hough.jpg")
    points_on_edge = []
    for x in range(thres_img.shape[0]):
        for y in range(thres_img.shape[1]):
            if thres_img[x][y] == 255:
                for k in range(len(edge_points)):
                    ro = hough(x,y,edge_points[k][0]-90)

                    if ro == edge_points[k][1]:
                        points_on_edge.append([x,y])
                        if edge_points[k][0] in range(80,95):
                            edge_img_vertical[x][y] = (255,0,0)
                        elif edge_points[k][0] in range(50,60):
                            edge_img_horizontal[x][y] = (0,255,0)
    cv2.imwrite("T3_Images/Vertical_Edge.jpg",edge_img_vertical)
    cv2.imwrite("T3_Images/Horizontal_Edge.jpg",edge_img_horizontal)
    

# Main

In [34]:
thres_img = np.zeros((sobelgx.shape[0], sobelgx.shape[1]))
x_coord = []
y_coord = []
for i in range(sobelgx.shape[0]):
    for j in range(sobelgx.shape[1]):
        if sobelgx[i][j] > 100:
            thres_img[i][j] = 255
cv2.imwrite("T3_Images/Updated_sobel.jpg",thres_img)

True

In [35]:
theta = []
for i in range(-90,90):
    theta.append(i)
theta = np.asarray(theta)

In [36]:
acc = accumulator_matrix(thres_img,theta)
edge_points = max_indices(acc, 100)
get_edge(edge_points)
